Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "" # put your full name here
COLLABORATORS = [] # list anyone you collaborated with on this workbook

---

## Lab 8: Resampling
**This lab was distributed the week of October 19, 2020 and should be completed by Tuesday 10/27/2020 at 11:59PM.**

Welcome to the eighth lab of the semester!

In this notebook, we'll explore resampling (which is relevant to Homework 8). Resampling is just what it sounds like - it involves repeatedly taking different samples of the data to train and test your model. We'll focus on cross-validation.

### Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore', FutureWarning) # Seaborn triggers warnings in scipy
%matplotlib inline

In [ ]:
# Configure nice plotting defaults - (this must be done in a cell separate from %matplotlib call)
plt.style.use('seaborn')
sns.set_context('talk', font_scale=1)
plt.rcParams['figure.figsize'] = (10, 6)

## Warm-up

Bootstrapping is a way to estimate the error on the coefficients of your model. It involves dividing your data into $B$ groups of $n$ observations (where $n$ is your sample size), then calculating the model coefficients for each of those groups. In total, you'll have $B$ different models, called bootstrap estimates. From there, you can find the standard error of your estimates. In bootstrapping, your $n$ observations are sampled with replacement - that means that one observation will appear in multiple bootstrap groups, but it also means that one bootstrap group can include the same observation more than once. A good source for more information about bootstrapping is ISLR 5.2.<br>

Suppose that we obtain a bootstrap sample from a set of $n$ observations. When $n$ = 5, what is the probability that the jth observation is in the bootstrap sample? What about when $n$=100?

*Your answer here*

----

## Creating Testing and Training Splits

To gain a little more intuition about cross-validation, we're going to work with the Boston Housing dataset, which concerns the housing values in the suburbs of Boston. We'll be using two features: `NOX`, the nitrogen oxides concentrations (in ppm) and `LSTAT`, the lower status of the population to predict the `target` column, the median value of owner-occupied homes (in thousands of dollars). Run the following cells to load the data and visualize the two features we'll be working with. 

In [ ]:
cv_data = pd.read_csv('data/boston_housing.csv')
cv_data.head()

In [ ]:
plt.scatter(cv_data.NOX, cv_data.target)
plt.xlabel('Nitrogen oxide concentration (ppm)')
plt.ylabel('Median value of owner-occupied homes (thousand $)');

In [ ]:
plt.scatter(cv_data.LSTAT, cv_data.target)
plt.xlabel('Lower status of the population')
plt.ylabel('Median value of owner-occupied homes (thousand $)');

Before we attempt cross validation, we're going to split our data into training and testing sets and fit a model to which we can compare our cross validation results. 

**Question 1** 

Split the dataset, using `NOX` and `LSTAT` as our features and `target` as what we're predicting. Set `test_size` to 25% of the sample and `random_state` to 2. Here, we'll use the [train_test_split() function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
# SOLUTION 
from sklearn.model_selection import train_test_split
    
X = ...
y = ...

X_train, X_test, y_train, y_test = ...

In [ ]:
assert np.isclose(len(X_train), 0.75* len(cv_data), atol=1)
assert np.isclose(len(y_test), 0.25*len(cv_data), atol=1)

Before we continue, let's review the arguments for `train_test_split`.

**Question 2** How do the parameters `test_size` and `random_state` affect the data we work with? How would a higher or lower value for `test_size` affect our results?

*Your answer here*

Now, let's fit the model with our training data.

**Question 3** Instantiate a `LinearRegression` model and fit the training data. Then, predict the `target` variable using the testing data. Lastly, print the $R^2$ score as well as the MSE of the fitted model. Here, we're using two new sklearn functions: [r2_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) and [mean_squared_error()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html).

In [ ]:
# SOLUTION 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

lm = ...
lm.fit(...)
predictions = 

print('R^2: ', r2_score(...))
print('MSE: ', mean_squared_error(...))

With these scores, we have a baseline with which to compare a cross-validated scores.

## Resampling and Cross-Validation

To recap, in the previous section we took a random sample of the data for training and a random sample for testing, and evaluated how the model, which was trained on the training data, performed on the testing data. But when we're trying to figure out how well our model will perform with new data, it's often not enough to get the MSE from just one random split into training and testing data. When we do a single split and train the model on a random subset of the data, we run the risk of over- or under-estimating the model error based on which observations made it into the training sets.

Enter cross-validation! 

<img src='cv.png' width="50%" height="50%"></img>

Let's discuss what we see in the figure above. We are given a dataset to work with, and do our usual train-test split. We will set aside the test data until we are absolutely ready to test the model we have created. However, in order to make sure that the model works as planned on data it is not 100% familiar with, we use cross-validation. In the image above, we split the training data into 4-folds. Essentially, each V that you see above represents a quarter of the training data set. This then acts as the validation set, and the model is fitted on the remaining 3/4 of the training set. This process is repeated three more times on different subsets, and the average MSE indicates the performance of the algorithm on the "unseen" data.

### Leave-One-Out Cross Validation

Let's begin by implementing Leave-One-Out Cross Validation (LOOCV), which essentially is $n$-fold cross validation, where $n$ is the number of observations in the dataset. For each fold, LOOCV splits the dataset two parts: a single observation $(x_i, y_i)$ is used as the validation set, and the rest are used as the training set. 

**Question 4**

What is a drawback with using only one observation for the validation set? Would LOOCV have much utility when splitting large datasets? Explain.

*YOUR ANSWER HERE*

We'll use scikit-learn's `LeaveOneOut` function to split our dataset. Run the following cell to perform LOOCV and check what happens to the $R^2$ score and MSE.

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
loo.get_n_splits(X)
y_tests = []
y_predictions = []
for train, test in loo.split(X): #a
    Xr_train, Xr_test = np.array(X)[train], np.array(X)[test] #b
    yr_train, yr_test = np.array(y)[train], np.array(y)[test] #c
    
    lr = LinearRegression()
    lr.fit(Xr_train, yr_train) 
    yr_pred = lr.predict(Xr_test) #d
    
    y_tests.append(yr_test) #e
    y_predictions.append(yr_pred) #e

r2_loo = r2_score(y_tests, y_predictions) 
MSE_loo = mean_squared_error(y_tests, y_predictions)
print("R^2 after LOOCV: ", r2_loo)
print("MSE after LOOCV: ", MSE_loo)

**Question 5** Several of the lines in the code are associated with a commented letter (e.g., *#a*). Answer the question associated with each letter:

a. For each iteration of the `for` loop, what is `test`, and what is `train`? How do `test` and `train` relate to each other, and how do they change over each iteration?<br>
b. What do the values of `Xr_train` represent? <br>
c. What do the values of `yr_test` represent?<br>
d. How many values are in the `y_pred` array?<br>
e. How many values are in the `y_test` and `y_predictions` array?<br>

In [ ]:
# scratch work

*YOUR ANSWER HERE*

**Question 6** Relative to Question 3, how does the $R^2$ score and MSE change when we do LOOCV?

*Your answer here*

**Question 7** Let's try one more experiment in comparing a simple training-testing approach with LOOCV. When we tried dividing the data into training and testing data in Question 1, we chose `random_state = 2` so that we would all end up with the same training and testing sets. Let's use the same approach to splitting our data and fitting a model as we inplemented in the previous section, but this time, we will vary `random_state`. We want to compare how the resulting $R^2$ and MSE generated from different random splits compare to the metrics obtained when we conducted LOOCV. Fill in the ellipses below, and then run following cells to generate a plot that shows the distribution of MSE and $R^2$ values using the different training-testing sets. In the plots, the MSE and $R^2$ values obtained from LOOCV are overlaid as vertical lines.

In [ ]:
ntrial = 100

r2 = []
MSE = []

for i in range(0,ntrial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=..)
    
    lm = ...
    lm.fit(...)
    predictions = lm.predict(...)

    r2.append(...)
    MSE.append(...)

In [ ]:
# run this cell
plt.figure(figsize = (20,5))

plt.subplot(121)
sns.kdeplot(MSE)
plt.axvline(x = MSE_loo, linewidth=4, color='r')
plt.xlabel("MSE")

plt.subplot(122)
sns.kdeplot(r2)
plt.axvline(x = r2_loo, linewidth=4, color='r')
plt.xlabel("$R^2$");

**Question 8** Where does the MSE and $R^2$ value from LOOCV sit relative to the distribution of values from the randomized training-testing approach? Does this result make sense to you given what you know about LOOCV? Why or why not? How might these graphs help you interpret the performance metrics you obtained in Question 3 to those in you obtained in Question 5?

*Your answer here*

### K-Fold Cross Validation with scikit-learn

We'll close by introducing how to implement k-fold cross validation using `scikit-learn`. Unlike LOOCV, k-fold cross validation uses a smaller number of folds than the number of observations in the dataset. 

**Question 9** Name one advantage and one disadvantage of k-fold cross validation relative to LOOCV.

*YOUR ANSWER HERE*

Let's practice splitting training data into k-folds for validation purposes. First, we'll import the `KFold` module from `scikit-learn`. 

In [ ]:
from sklearn.model_selection import KFold

Now we'll split the array `X` from Question 1 into 5 folds, shuffling before we add the batches, with a random state of 1. For each fold, we'll print the indices of the Train and Validation sets onto the console.

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

fold = 1
for train_i, val_i in kf.split(X):
    print("Fold", fold)
    print("Train:", train_i, "Test:", val_i, '\n')
    fold+=1

In the homework, you'll have a chance to build upon this framework to create a k-fold cross validation pipeline to systematically split your data, fit a model using the training data, and compute the average MSE and $R^2$. 

### Bibliography
- DS100 - “Gradient Descent” - https://www.textbook.ds100.org/ch/11/gradient_descent_define.html 
- DS100 - “Absolute Loss” - https://www.textbook.ds100.org/ch/10/modeling_abs_huber.html
- DS100 - “Models and Estimation” - http://www.ds100.org/fa18/assets/lectures/lec09/09-Models-and-Estimation-II.html 

---
Notebook developed by: Joshua Asuncion, Rebekah Tang

Data Science Modules: http://data.berkeley.edu/education/modules
